In [ ]:
import os
import sqlalchemy as sa
from sqlalchemy.orm import sessionmaker, declarative_base
from cryptography.fernet import Fernet, InvalidToken
from dotenv import load_dotenv

class EncryptedDatabase:
    # Load environment variables and define constants
    load_dotenv('backend/db/db_paths.env')

    DATABASE_URL = os.getenv("API_KEYS_DB_PATH")
    KEY_FILE = "encryption_key.key"

    # Initialize database engine, session, and ORM base
    engine = sa.create_engine(DATABASE_URL, echo=True)
    Session = sessionmaker(bind=engine)
    Base = declarative_base()
    cipher = None  # Placeholder for encryption cipher

    # Database Model
    class APIKeyStore(Base):
        __tablename__ = "api_keys"

        id = sa.Column(sa.Integer, primary_key=True)
        name = sa.Column(sa.String, unique=True, nullable=False)
        api_key = sa.Column(sa.String, nullable=False)
        secret_key = sa.Column(sa.String, nullable=True)
        api_metadata = sa.Column(sa.String, nullable=True)

        def __init__(self, name, api_key, secret_key=None, api_metadata=None):
            self.name = name
            self.api_key = EncryptedDatabase.cipher.encrypt(api_key.encode()).decode()
            self.secret_key = EncryptedDatabase.cipher.encrypt(secret_key.encode()).decode() if secret_key else None
            self.api_metadata = api_metadata

        def decrypt_data(self):
            try:
                self.api_key = EncryptedDatabase.cipher.decrypt(self.api_key.encode()).decode()
                if self.secret_key:
                    self.secret_key = EncryptedDatabase.cipher.decrypt(self.secret_key.encode()).decode()
            except InvalidToken:
                raise ValueError("❌ Decryption failed. Invalid encryption key.")

    @classmethod
    def load_or_generate_key(cls):
        """Loads the encryption key from a file or generates a new one."""
        if not os.path.exists(cls.KEY_FILE):
            key = Fernet.generate_key()
            with open(cls.KEY_FILE, "wb") as key_file:
                key_file.write(key)
            print("🔑 New encryption key generated and saved.")
        else:
            print("✅ Encryption key already exists.")

        # Load the key
        with open(cls.KEY_FILE, "rb") as key_file:
            return key_file.read()

    @classmethod
    def init_cipher(cls):
        """Initializes the encryption cipher."""
        cls.cipher = Fernet(cls.load_or_generate_key())

    @classmethod
    def init_db(cls):
        """Creates the database if it doesn't exist, otherwise does nothing."""
        db_file = cls.DATABASE_URL.replace("sqlite:///", "")
        if os.path.exists(db_file):
            print("✅ Database already exists.")
        else:
            print("📦 Creating database...")
            cls.Base.metadata.create_all(cls.engine)
            print("✅ Database initialized.")

    @classmethod
    def insert_api_key(cls, name, api_key, secret_key=None, api_metadata=None):
        """Inserts a new API key into the database."""
        session = cls.Session()
        if session.query(cls.APIKeyStore).filter_by(name=name).first():
            print(f"⚠️ API Key with name '{name}' already exists.")
            session.close()
            return
        new_key = cls.APIKeyStore(name=name, api_key=api_key, secret_key=secret_key, api_metadata=api_metadata)
        session.add(new_key)
        session.commit()
        session.close()
        print(f"✅ API Key '{name}' added successfully.")

    @classmethod
    def get_api_keys(cls):
        """Retrieves and decrypts API keys from the database."""
        session = cls.Session()
        keys = session.query(cls.APIKeyStore).all()
        for key in keys:
            key.decrypt_data()
        session.close()
        return keys


# Initialize encryption and database
EncryptedDatabase.init_cipher()
EncryptedDatabase.init_db()

# Example usage
EncryptedDatabase.insert_api_key("abc", "abcdeffg", "s3cr3t")

# Retrieve and print decrypted API keys
for key in EncryptedDatabase.get_api_keys():
    print(f"🔑 Name: {key.name}, API Key: {key.api_key}, Secret Key: {key.secret_key}, Metadata: {key.api_metadata}")